In [93]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [95]:
df=pd.read_csv('/content/drive/MyDrive/shuttle-c2-vs-c4.csv')
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,52,-30,79,0,0,0,27,79,52,positive
1,90,-40,107,0,64,-28,17,42,26,positive
2,70,-30,107,0,60,-13,37,47,10,positive
3,71,-33,107,0,60,-21,36,47,12,positive
4,37,-85,77,0,20,0,40,57,16,positive
...,...,...,...,...,...,...,...,...,...,...
124,75,0,80,4,-42,0,5,124,120,negative
125,85,0,88,0,2,-17,3,86,82,negative
126,81,0,86,0,-40,2,4,127,122,negative
127,84,0,88,0,-2,-12,3,90,88,negative


In [96]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0      positive
1      positive
2      positive
3      positive
4      positive
         ...   
124    negative
125    negative
126    negative
127    negative
128    negative
Name: Class, Length: 129, dtype: object

In [97]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,52,-30,79,0,0,0,27,79,52,1
1,90,-40,107,0,64,-28,17,42,26,1
2,70,-30,107,0,60,-13,37,47,10,1
3,71,-33,107,0,60,-21,36,47,12,1
4,37,-85,77,0,20,0,40,57,16,1


In [98]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(6, 9)

In [99]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(123, 9)

In [100]:
df_minor = pd.DataFrame(minority, columns =['A1','A2','A3','A4','A5','A6','A7','A8','A9'])
df_minor

,A1,A2,A3,A4,A5,A6,A7,A8,A9
0,52.000000,-30.000000,79.0,0.0,0.000000,0.0,27.0,79.0,52.000000
1,90.000000,-40.000000,107.0,0.0,64.000000,-28.0,17.0,42.0,26.000000
2,70.000000,-30.000000,107.0,0.0,60.000000,-13.0,37.0,47.0,10.000000
3,71.000000,-33.000000,107.0,0.0,60.000000,-21.0,36.0,47.0,12.000000
4,37.000000,-85.000000,77.0,0.0,20.000000,0.0,40.0,57.0,16.000000
...,...,...,...,...,...,...,...,...,...
118,58.103645,-61.381559,107.0,0.0,64.000000,0.0,40.0,79.0,30.550762
119,90.000000,-30.000000,107.0,0.0,59.940453,0.0,40.0,79.0,52.000000
120,63.547770,-30.000000,107.0,0.0,64.000000,0.0,17.0,79.0,52.000000
121,90.000000,-85.000000,107.0,0.0,64.000000,0.0,40.0,79.0,52.000000


In [101]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,52.000000,-30.000000,79.0,0.0,0.000000,0.0,27.0,79.0,52.000000,1
1,90.000000,-40.000000,107.0,0.0,64.000000,-28.0,17.0,42.0,26.000000,1
2,70.000000,-30.000000,107.0,0.0,60.000000,-13.0,37.0,47.0,10.000000,1
3,71.000000,-33.000000,107.0,0.0,60.000000,-21.0,36.0,47.0,12.000000,1
4,37.000000,-85.000000,77.0,0.0,20.000000,0.0,40.0,57.0,16.000000,1
...,...,...,...,...,...,...,...,...,...,...
118,58.103645,-61.381559,107.0,0.0,64.000000,0.0,40.0,79.0,30.550762,1
119,90.000000,-30.000000,107.0,0.0,59.940453,0.0,40.0,79.0,52.000000,1
120,63.547770,-30.000000,107.0,0.0,64.000000,0.0,17.0,79.0,52.000000,1
121,90.000000,-85.000000,107.0,0.0,64.000000,0.0,40.0,79.0,52.000000,1


In [102]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','Class'])
df_major

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,81,-5,84,0,-22,-6,4,108,104,0
1,103,-1,104,-4,72,10,1,31,30,0
2,86,5,89,0,2,-5,3,86,84,0
3,106,0,108,-3,70,-4,1,38,36,0
4,79,0,83,0,-46,-29,4,130,126,0
...,...,...,...,...,...,...,...,...,...,...
118,75,0,80,4,-42,0,5,124,120,0
119,85,0,88,0,2,-17,3,86,82,0
120,81,0,86,0,-40,2,4,127,122,0
121,84,0,88,0,-2,-12,3,90,88,0


In [103]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,Class
0,52.0,-30.0,79.0,0.0,0.0,0.0,27.0,79.0,52.0,1
1,90.0,-40.0,107.0,0.0,64.0,-28.0,17.0,42.0,26.0,1
2,70.0,-30.0,107.0,0.0,60.0,-13.0,37.0,47.0,10.0,1
3,71.0,-33.0,107.0,0.0,60.0,-21.0,36.0,47.0,12.0,1
4,37.0,-85.0,77.0,0.0,20.0,0.0,40.0,57.0,16.0,1
...,...,...,...,...,...,...,...,...,...,...
241,75.0,0.0,80.0,4.0,-42.0,0.0,5.0,124.0,120.0,0
242,85.0,0.0,88.0,0.0,2.0,-17.0,3.0,86.0,82.0,0
243,81.0,0.0,86.0,0.0,-40.0,2.0,4.0,127.0,122.0,0
244,84.0,0.0,88.0,0.0,-2.0,-12.0,3.0,90.0,88.0,0


In [104]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [105]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [106]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [1.0, 1.0, 1.0, 1.0, 1.0]
AUC_SVM: [1.0, 1.0, 1.0, 1.0, 1.0]
f1_SVM: [1.0, 1.0, 1.0, 1.0, 1.0]
precision_SVM: [1.0, 1.0, 1.0, 1.0, 1.0]
recall_SVM: [1.0, 1.0, 1.0, 1.0, 1.0]
mean of Gmean_s: 1.0
mean of AUC_s: 1.0
mean of f1_s: 1.0
mean of precision_s: 1.0
mean of recall_s: 1.0
standard deviation of Gmean_s: 0.0
standard deviation of AUC_s: 0.0
standard deviation of f1_s: 0.0
standard deviation of precision_s: 0.0
standard deviation of recall_s: 0.0


-------KNN--------
Gmean_KNN: [1.0, 1.0, 1.0, 1.0, 1.0]
AUC_KNN: [1.0, 1.0, 1.0, 1.0, 1.0]
f1_KNN: [1.0, 1.0, 1.0, 1.0, 1.0]
precision_KNN: [1.0, 1.0, 1.0, 1.0, 1.0]
recall_KNN: [1.0, 1.0, 1.0, 1.0, 1.0]
mean of Gmean_k: 1.0
mean of AUC_k: 1.0
mean of f1_k: 1.0
mean of precision_k: 1.0
mean of recall_k: 1.0
standard deviation of Gmean_k: 0.0
standard deviation of AUC_k: 0.0
standard deviation of f1_k: 0.0
standard deviation of precision_k: 0.0
standard deviation of recall_k: 0.0


-------DECISION TREE--------

In [110]:
list=['shuttle2vs4',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0.1,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev
2,ecoli0137vs26,0.959495,0.040136,0.92449,0.072456,0.959798,0.027859,0.923228,0.052104,0.946633,0.061364,0.905228,0.105836
3,shuttle0vs4,0.999414,0.000802,0.998826,0.001607,1.0,0.0,1.0,0.0,0.998828,0.000655,0.997651,0.001313
4,yeast1vs7,0.953475,0.075796,0.923035,0.116982,0.938317,0.063328,0.89309,0.093762,0.963953,0.080602,0.94367,0.125957


In [111]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')